# Problem definition

We wish to minimize
$$ I(u,v) = \frac{\theta}{2} \int_{\omega} |\nabla_s u + \tfrac{1}{2} \nabla v \otimes \nabla v|^{2} \mathrm{d}x
   + \frac{1}{24} \int_{\omega} |\nabla^2 v - \mathrm{Id}|^{2} \mathrm{d}x. $$

Because we only have $C^0$ elements we set $z$ for $\nabla v$ and minimize instead

$$ J(u,z) = \frac{\theta}{2} \int_{\omega} |\nabla_s u + \tfrac{1}{2} z \otimes z|^{2} \mathrm{d}x 
          + \frac{1}{24} \int_{\omega} |\nabla z - \mathrm{Id}|^{2} \mathrm{d}x 
          + \mu \int_{\omega} |\mathrm{curl}\ z|^{2} \mathrm{d}x, $$

then recover the vertical displacements (up to a constant) by minimizing

$$ F(p,q) = \tfrac{1}{2} || \nabla p - q ||^2 + \tfrac{1}{2} || q - z ||^2. $$

This we do by solving the linear problem $D F = 0$.

Minimization of the energy functional $J$ is done via gradient descent and a line search. In particular, at each timestep we compute $d_t w \in W $ such that for all $\tau \in W$:

$$ (d_t w, \tau)_{H^1_0 \times H^2_0} = -DJ(w_t)[\tau] $$

Note that the it is essential to use the full scalar product (or the one corresponding to the seminorms? check this) or we run into issues at the boundaries (to see this start with zero displacements and integrate by parts).(Also: the proper Riesz representative will only be obtained with correct scalar product).

A decoupled gradient descent in each component does not work, probably because the functional is not separately convex (see Bartels' book, p. 110, remark (iv)).

In plane displacements and gradients of out of plane displacements form a mixed function space $U \times Z$. We also have another scalar space $V$ where the potential of the out of plane gradients lives. The model is defined and solved in `run_model()` below. Experiments can be easily run in parallel with `joblib`.

Another quadratic form (isotropic material):

$$ Q_2 (F) = 2 \mu | \operatorname{sym} F |^2 + \frac{2 \mu \lambda}{2 \mu + \lambda}
   \operatorname{tr}^2 F, \quad F \in \mathbb{R}^{2 \times 2} . $$

In [ ]:
from dolfin import *
import os
import mshr
import numpy as np
import matplotlib.pyplot as pl
from time import time
from tqdm import tqdm_notebook as tqdm
from common import *

In [ ]:
def run_model(init: str, qform: str, mesh_file: str, theta: float, mu: float = 0.0,
              e_stop_mult: float = 1e-5, max_steps: int = 400, save_funs: bool = True, n=0):
    """
    """

    qform = qform.lower()

    msh = Mesh(mesh_file)

    t = tqdm(total=max_steps, desc='th=% 8.3f' % theta, position=n, dynamic_ncols=True)

    def noop(*args, **kwargs):
        pass

    def tout(s, **kwargs):
        """ FIXME: Does not work as intended... """
        t.write(s, end='')

    debug = noop
    # debug = print

    # in plane displacements (IPD)
    UE = VectorElement("Lagrange", msh.ufl_cell(), 2, dim=2)
    # Gradients of out of plane displacements (OPD)
    VE = VectorElement("Lagrange", msh.ufl_cell(), 2, dim=2)
    W = FunctionSpace(msh, UE * VE)
    # will store out of plane displacements
    V = FunctionSpace(msh, "Lagrange", 2)

    # We gather in-plane and out-of-plane displacements into one
    # Function for visualization with ParaView.
    P = VectorFunctionSpace(msh, "Lagrange", 2, dim=3)
    fax = FunctionAssigner(P.sub(0), W.sub(0).sub(0))
    fay = FunctionAssigner(P.sub(1), W.sub(0).sub(1))
    faz = FunctionAssigner(P.sub(2), V)

    disp = Function(P)
    disp.rename("disp", "displacement")

    file_name = make_filename('curl', init, qform, theta, mu, create_dir=True)
    file = File(file_name)  # .vtu files will have the same prefix

    w = Function(W)
    w_ = Function(W)
    u, v = w.split()
    u_, v_ = w_.split()

    w_init = make_initial_data_penalty(init)
    w.interpolate(w_init)
    w_.interpolate(w_init)

    if qform == 'frobenius':
        Q2, L2 = frobenius_form()
    elif qform == 'isotropic':
        # Isotropic density for steel at room temp.
        # E is in GPa. Is it ok to use these units? Setting it to 210e9
        # breaks things (line searches don't end) because we need to scale
        # elastic constants with h
        E, nu = 210.0, 0.3
        Q2, L2 = isotropic_form(E * nu / ((1 + nu) * (1 - 2 * nu)), E / (2 + 2 * nu))
    else:
        raise Exception("Unknown quadratic form name '%s'" % qform)

    def eps(u):
        return (grad(u) + grad(u).T) / 2.0

    e_stop = msh.hmin() * e_stop_mult
    max_line_search_steps = 20
    step = 0
    omega = 0.25  # Gradient descent fudge factor in (0, 1/2)
    _hist = {'init': init, 'impl': 'curl', 'mu': mu, 'theta': theta, 'e_stop': e_stop,
             'J': [], 'alpha': [], 'du': [], 'dv': [], 'constraint': [],
             'Q2': {'form_name': Q2.__name__, 'arguments': Q2.arguments},
             'symmetry': [], 'file_name': file_name}

    B = Identity(2)
    zero_energy = assemble((1. / 24) * inner(B, B) * dx(msh))

    def energy(u, v, mu=mu):
        J = (theta / 2.) * Q2(eps(u) + outer(v, v) / 2) * dx(msh) \
            + (1. / 24) * Q2(grad(v) - B) * dx(msh) \
            + mu * inner(curl(v), curl(v)) * dx(msh)
        return assemble(J)

    # CAREFUL!! Picking the right scalar product here is essential
    # Recall the issues with boundary values: integrate partially and only boundary terms survive...
    dtu, dtv = TrialFunctions(W)
    phi, psi = TestFunctions(W)
    L = inner(dtu, phi) * dx + inner(grad(dtu), grad(phi)) * dx \
        + inner(dtv, psi) * dx + inner(grad(dtv), grad(psi)) * dx

    dw = Function(W)
    du, dv = dw.split()

    # Output initial condition
    opd = compute_potential(v, V)
    fax.assign(disp.sub(0), u.sub(0))
    fay.assign(disp.sub(1), u.sub(1))
    faz.assign(disp.sub(2), opd)
    file << (disp, float(step))

    cur_energy = energy(u, v)
    alpha = ndu = ndv = 1.0

    debug("Solving with theta = %.2e, mu = %.2e, eps=%.2e for at most %d steps."
          % (theta, mu, e_stop, max_steps))

    begin = time()
    while alpha * (ndu ** 2 + ndv ** 2) > e_stop and step < max_steps:
        _curl = assemble(curl(v_) * dx)
        _symmetry = circular_symmetry(disp)
        _hist['constraint'].append(_curl)
        _hist['symmetry'].append(_symmetry)
        debug("Step %d, energy = %.3e, curl = %.3e, symmetry = %.3f"
              % (step, cur_energy, _curl, _symmetry))

        #### Gradient
        # for some reason I'm not able to use derivative(J, w_, dtw)
        dJ = theta * L2(eps(u_) + outer(v_, v_) / 2,
                        eps(phi) + sym(outer(v_, psi))) * dx(msh) \
             + (1. / 12) * L2(grad(v_) - B, grad(psi)) * dx(msh) \
             + 2 * mu * inner(curl(v_), curl(psi)) * dx(msh)

        debug("\tSolving...", end='')
        solve(L == -dJ, dw, [])

        du, dv = dw.split()
        # dw is never reassigned to a new object so it's ok
        # to reuse du, dv without resplitting
        ndu = norm(du)
        ndv = norm(dv)

        debug(" done with |du| = %.3f, |dv| = %.3f" % (ndu, ndv))

        #### Line search
        new_energy = 0
        debug("\tSearching... ", end='')
        while True:
            w = project(w_ + alpha * dw, W)
            u, v = w.split()
            new_energy = energy(u, v)
            if new_energy <= cur_energy - omega * alpha * (ndu ** 2 + ndv ** 2):
                debug(" alpha = %.2e" % alpha)
                _hist['J'].append(cur_energy)
                _hist['alpha'].append(alpha)
                _hist['du'].append(ndu)
                _hist['dv'].append(ndv)
                cur_energy = new_energy
                alpha = min(1.0, 2.0 * alpha)  # Use a larger alpha for the next line search
                break
            if alpha < (1. / 2) ** max_line_search_steps:
                raise Exception("Line search failed after %d steps" % max_line_search_steps)
            alpha /= 2.0  # Repeat with smaller alpha

        step += 1

        #### Write displacements to file
        debug("\tSaving... ", end='')
        opd = compute_potential(v, V)
        fax.assign(disp.sub(0), u.sub(0))
        fay.assign(disp.sub(1), u.sub(1))
        faz.assign(disp.sub(2), opd)
        file << (disp, float(step))
        debug("Done.")

        w_.vector()[:] = w.vector()
        u_, v_ = w_.split()
        t.update()

    _hist['time'] = time() - begin

    if step < max_steps:
        t.total = step
        t.update()

    _hist['steps'] = step
    if save_funs:
        _hist['disp'] = disp
        _hist['u'] = u
        _hist['v'] = v
        _hist['dtu'] = du
        _hist['dtv'] = dv
    debug("Done after %d steps" % step)

    t.close()
    return _hist


For exploration in this notebook, we store outputs from different runs in a global array

In [ ]:
if globals().get('history') is None:
    history = []

In [ ]:
parameters["form_compiler"]["optimize"]     = True
parameters["form_compiler"]["cpp_optimize"] = True

mesh_file = generate_mesh('circle', 18, 18)
theta = 10.0

_hist = run_model('ani_parab', mesh_file, theta=theta, mu=0.0,
                  max_steps=10000, save_funs=True, e_stop_mult=1e-7)

history.append(_hist)

# Results

In [ ]:
plots1(history[-1], slice(0,-1))

# Solution and last update

In [ ]:
plots2(history[-1])

# Exploring the range of $\theta$

In [ ]:
from tqdm import tqdm
from joblib import Parallel, delayed

theta_values = np.arange(0.0, 100.0, 1.0, dtype=float)
# Careful: hyperthreading won't help (we are probably bound by memory r/w)
n_jobs = min(11, len(theta_values))

new_res = Parallel(n_jobs=n_jobs)(delayed(run_model)('ani_parab', theta=theta, mu=0.0,
                                                     max_steps=2000, save_funs=False,
                                                     e_stop_mult=1e-6, n=n) 
                                  for n, theta in enumerate(theta_values))

save_results(new_res, "results-combined.pickle")

With increasing $\theta$ we expect the symmetry of the solution to be ever more violated until it is cylindrical rather than parabolic. However there seems to be no clear discontinuity. This can be due to 

* a poor criterion for symmetry (we are just taking the quotient of the principal axes)
* solutions not being proper minima (gradient descent didn't converge to $\epsilon_{\text{stop}}$ precision)
* ...

In [ ]:
%matplotlib tk
from plots import *

import matplotlib.pyplot as pl
import pickle as pk

with open("results-curl-combined.pickle", "rb") as f:
    res = pk.load(f)

from common import *
import os

In [ ]:
with open("results-curl-combined.pickle", "wb") as f:
    pk.dump(res, f)

Filter runs prematurely stopped and with high values of $\epsilon_{\text{stop}}$.

In [ ]:
all_eps = set(v['e_stop'] for k, v in res.items())

fres = {k:v for k, v in res.items() if v['e_stop'] == min(all_eps)}
fres2 = {k:v for k, v in fres.items() if v['steps'] not in [500, 2000] }

In [ ]:
plots3(fres, 0, 1000)

Plot data about some weird runs:

In [ ]:
plots1(get_longest(fres, 8.7), slice(10, -1), 20)

In [ ]:
plots4(filter_results(fres, theta=(8,9)), slice(20, -1), 20)

# A simple report

This uses [yattag](http://www.yattag.org) for the html generation. The report itself depends on [Footable](http://fooplugins.github.io/FooTable/index.html), which uses jQuery and FontAwesome. This is all quite unnecessary.

In [ ]:
from yattag import Doc, indent


def make_report(res:dict):
    doc, tag, text, line = Doc().ttl()

    doc.asis('<!DOCTYPE html>')
    with tag('html', lang='en'):
        with tag('head'):
            line('title', 'Results')
            line('meta', '', charset='utf-8')
            line('link', '', rel='stylesheet', href='css/footable.standalone.min.css')
            line('link', '', rel='stylesheet', href='css/font-awesome.min.css')
        with tag('body'):
            with tag('table', ('data-sorting', 'true'), ('data-filtering', 'true'), ('data-show-toggle', 'false'),
                     klass="table"):
                with tag('thead'):
                    with tag('tr'):
                        line('th', 'form_name')
                        line('th', 'init')
                        line('th', 'theta', ('data-type', 'number'))
                        line('th', 'mu', ('data-type', 'number'))
                        line('th', 'eps magn.', ('data-type', 'number'))
                        line('th', 'steps', ('data-type', 'number'))
                        line('th', 'duration', ('data-type', 'time'))
                        line('th', 'plot', ('data-type', 'html'))
                        line('th', 'results', ('data-type', 'html'),
                             ('data-breakpoints', 'all'), ('data-title', 'results file:'))
                        line('th', 'form arguments',
                             ('data-breakpoints', 'all'), ('data-title', 'form arguments:'))
                with tag('tbody'):
                    for k, v in res.items():
                        with tag('tr'):
                            line('td', v['Q2']['form_name'])
                            line('td', v['init'])
                            line('td', v['theta'])
                            line('td', v['mu'])
                            line('td', int(np.log10(v['e_stop'])))
                            line('td', v['steps'])
                            tt = max(0, v['time'])
                            hh = np.floor(tt/3600)
                            mm = np.floor((tt - hh*3600)/60)
                            ss = np.floor((tt - hh*3600 - mm*60))
                            line('td', "%02d:%02d:%02d" % (hh, mm, ss))
                            with tag('td'):
                                line('a', 'plot', href='../%s' % v['plots1_fname'])
                            line('td', v['file_name'])
                            line('td', str(v['Q2']['arguments']))
            line('script', '', src='js/jquery-3.2.1.min.js')
            line('script', '', src='js/footable.min.js')
            with tag('script'):
                doc.asis("jQuery(function($){$('.table').footable();});")
    with open("report/index.html", "wt") as fd:
        fd.write(indent(doc.getvalue()))

In [ ]:
make_report(res)

In [ ]:
plots4(filter_results(res, theta=(20,25)), slice(50, -1), 50)